In [1]:
import keras
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import *

/home/jorgher/.virtualenvs/cv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = "/home/jorgher/Documents/faceRec/CFEE/"
files = sorted(glob.glob(path+'*.csv'))
files = files[:7]
emotions_ = ["neutral",'happy','sad','fearful','angry','surprised','disgusted']
emotions = [0,1,2,3,4,5,6]
#emotions = [0,1,2,3,4]
n_classes = len(emotions)
dbs = []
for i,f in enumerate(files):
    db = pd.read_csv(f)
    db['file'] = db['file'].map(lambda x: str(x)[15:])
    db['file'] = db['file'].map(lambda x: str(x)[:-4]).astype(int)
    db['Subjet'] = db['file']
    db['Emotions'] = emotions[i]
    dbs.append(db)

db = (pd.concat(dbs[i][:] for i in range(len(files))))
db = db.sort_values('file')
#df['Emotion']=df['file']
db.drop(['file'], axis=1, inplace=True)

In [3]:
features = list(db.columns[:13])
Etype={'neutral': 0, 'happy': 1,
        'sad': 2,'fearful': 3,'angry': 4,
        'surprised': 5,'disgusted': 6}
Etype_={0:'neutral', 1: 'happy',
         2:'sad', 3:'fearful', 4:'angry',
         5:'surprised', 6:'disgusted'}

In [4]:
db_data = db[features]
db_target = db['Emotions']
db_data.shape,db_target.shape

((1610, 13), (1610,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(db_data, db_target, test_size=0.3, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_t= scaler.transform(X_train)
y_train_t = keras.utils.to_categorical(y_train)
X_test_t = scaler.transform(X_test)
y_test_t = keras.utils.to_categorical(y_test)

In [6]:
X_train_t.shape, y_train_t.shape

((1127, 13), (1127, 7))

In [7]:
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 13-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=13))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [10]:
optimizers = [SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
              Adagrad(lr=0.01, epsilon=None, decay=0.0),
              Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0),
              Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0),#, amsgrad=False),
              Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0),
              Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)]#,TFOptimizer(optimizer)]

optimizers_ = ['SGD','RMSprop','Adagrad','Adadelta','Adam','Adamax','Nadam']
loss_ = ['mean_squared_error','mean_absolute_error','']

for i,opt in enumerate(optimizers_):
    print(' Optimizador: '+ opt)
    model.compile(loss='categorical_crossentropy',
              optimizer= opt,
              metrics=['accuracy'])
    model.fit(X_train_t, y_train_t,
              epochs=30,verbose=0,
              batch_size=128)
    
    (loss, accuracy) = model.evaluate(X_test_t, y_test_t,
                                      batch_size=128, verbose=0)
    print("La perdida es: {:.2f}, y la precision: {:.2f}%".format(loss,accuracy * 100))
    
    print("Grabando los coeficientes al archivo...")
    model.save_weights('Emo_weights'+opt+'.hdf5', overwrite=True)
   
    ## Testing with a set of vectors coming from facerec app.
    probs = model.predict([[ 0.39466814,  0.36386604,  0.37833949,  0.08193851,  0.08193851,  0.23204663,
   0.70771351,  0.01217944,  0.00351996,  0.0120441,   0.00504033,  0.00701828,
   0.01361792],[ 0.39466814,  0.36386604,  0.37833949,  0.08193851,  0.08193851,  0.23204663,
   0.70771351,  0.01217944,  0.00351996,  0.0120441,   0.00504033,  0.00701828,
   0.01361792],[ 0.39466814,  0.36386604,  0.37833949,  0.08193851,  0.08193851,  0.23204663,
   0.70771351,  0.01217944,  0.00351996,  0.0120441,   0.00504033,  0.00701828,
   0.01361792],[ 0.39466814,  0.36386604,  0.37833949,  0.08193851,  0.08193851,  0.23204663,
   0.70771351,  0.01217944,  0.00351996,  0.0120441,   0.00504033,  0.00701828,
   0.01361792],[ 0.39466814,  0.36386604,  0.37833949,  0.08193851,  0.08193851,  0.23204663,
   0.70771351,  0.01217944,  0.00351996,  0.0120441,   0.00504033,  0.00701828,
   0.01361792],[ 0.39466814,  0.36386604,  0.37833949,  0.08193851,  0.08193851,  0.23204663,
   0.70771351,  0.01217944,  0.00351996,  0.0120441,   0.00504033,  0.00701828,
   0.01361792],[ 0.39466814,  0.36386604,  0.37833949,  0.08193851,  0.08193851,  0.23204663,
   0.70771351,  0.01217944,  0.00351996,  0.0120441,   0.00504033,  0.00701828,
   0.01361792],[ 0.39466814,  0.36386604,  0.37833949,  0.08193851,  0.08193851,  0.23204663,
   0.70771351,  0.01217944,  0.00351996,  0.0120441,   0.00504033,  0.00701828,
   0.01361792],[ 0.39466814,  0.36386604,  0.37833949,  0.08193851,  0.08193851,  0.23204663,
   0.70771351,  0.01217944,  0.00351996,  0.0120441,   0.00504033,  0.00701828,
   0.01361792]])

    prediction = probs.argmax(axis=1)
    print("[INFO] Predicted: {}".format(prediction[0]))
    
    score = model.evaluate(X_test_t, y_test_t, verbose = 0, batch_size=32)
    print("La eficiencia del modelo es: {:.4f}, y su precision es: {:.4f}%".format(score[0],score[1]*100)) 
    print()
    print()

 Optimizador: SGD
La perdida es: 0.83, y la precision: 69.57%
Grabando los coeficientes al archivo...
[INFO] Predicted: 0
La eficiencia del modelo es: 0.8278, y su precision es: 69.5652%


 Optimizador: RMSprop
La perdida es: 0.84, y la precision: 69.77%
Grabando los coeficientes al archivo...
[INFO] Predicted: 0
La eficiencia del modelo es: 0.8370, y su precision es: 69.7723%


 Optimizador: Adagrad
La perdida es: 0.84, y la precision: 69.36%
Grabando los coeficientes al archivo...
[INFO] Predicted: 0
La eficiencia del modelo es: 0.8420, y su precision es: 69.3582%


 Optimizador: Adadelta
La perdida es: 0.84, y la precision: 69.36%
Grabando los coeficientes al archivo...
[INFO] Predicted: 0
La eficiencia del modelo es: 0.8399, y su precision es: 69.3582%


 Optimizador: Adam
La perdida es: 0.84, y la precision: 69.98%
Grabando los coeficientes al archivo...
[INFO] Predicted: 0
La eficiencia del modelo es: 0.8432, y su precision es: 69.9793%


 Optimizador: Adamax
La perdida es: 0.85,